In [ ]:
import warnings
warnings.simplefilter('ignore')

# EarthSim: easy two-way map interactions

In [ ]:
import geoviews as gv
import holoviews as hv
import cartopy.crs as ccrs

from holoviews.streams import PointDraw, PolyEdit, BoxEdit, PolyDraw

url = 'http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png'
dx = dy = 1
lat, lon = 47.60, -122.33
extents = lon-dx, lat-dy, lon+dx, lat+dy

tiles = gv.WMTS(url, extents=extents, crs=ccrs.PlateCarree())

hv.extension('bokeh')

### Drawing Points

- Add point: click to add
- Move point: drag an existing point
- Delete point: select and press the backspace/delete

In [ ]:
%%opts Points [width=750 height=500 tools=['hover']]
%%opts Points (size=10 color='red')

points = gv.Points([[-13616490, 6039761]], crs=ccrs.GOOGLE_MERCATOR)
point_stream = PointDraw(source=points)

tiles * points

In [ ]:
if point_stream.data:
    projected = gv.operation.project_points(point_stream.element, projection=ccrs.PlateCarree())
    display(projected.dframe())

### Drawing bounding boxes

- Add box: hold shift, click and drag
- Move box: click and drag an existing box
- Delete box: click and press the backspace/delete

In [ ]:
%%opts Polygons [width=750 height=500] 
%%opts Polygons (fill_alpha=0 line_color='black' selection_fill_color='red' line_width=5)

sample_box = [[-13873713, 5925064], [-13873713, 6348504],
              [-13461934, 6348504], [-13461934, 5925064],]

box_poly = gv.Polygons([sample_box], crs=ccrs.GOOGLE_MERCATOR)
box_stream = BoxEdit(source=box_poly)
tiles * box_poly

In [ ]:
def bbox(poly):
    'Convert the polygon returned by the BoxEdit stream into a bounding box tuple'
    xs,ys = poly.array().T
    return (xs[0], ys[0], xs[2], ys[2])

if box_stream.element:
    polygons = gv.operation.project_path(box_stream.element, 
                                         projection=ccrs.PlateCarree()).split()
    bboxes = [bbox(p) for p in polygons]
    print(bboxes)

### Drawing Polygons

- Add patch/multi-line: double click to add the first vertex, then use tap to add each subsequent vertex. To finalize the draw action, double tap to insert the final vertex or press the ESC key to stop drawing.
- Move patch/multi-line: drag an existing object
- Delete: select and press backspace/delete

In [ ]:
%%opts Polygons [width=750 height=500] (fill_alpha=0.1 line_color='black') 
%%opts Path (line_width=5 color='black')

sample_poly = {
    'Longitude': [-13873713, -13461934, -13873713],
    'Latitude': [6348504, 5925064, 5925064],
}
new_polys = gv.Polygons([sample_poly], crs=ccrs.GOOGLE_MERCATOR)
poly_stream = PolyDraw(source=new_polys)

sample_path = {
    'Longitude': [-13873713, -13461934],
    'Latitude': [6348504, 5925064],
}

new_paths = gv.Path([sample_path], crs=ccrs.GOOGLE_MERCATOR)
path_stream = PolyDraw(source=new_paths)

tiles * new_polys * new_paths

### Polygon Editing

- Show vertices: double click an existing object
- Add vertex: click an existing vertex, then the tool will draw the next point; to add it tap in a new location. To finish editing and add a point, double tap; otherwise press the ESC key to cancel.
- Move vertex: drag an existing vertex
- Delete vertex: selecting press backspace/delete

In [ ]:
from pathlib import Path

path = Path('data')
shapefile = str(path.joinpath('model_grid', 'grid.shp'))

In [ ]:
%%opts Shape [width=750 height=500 tools=['box_select']] (alpha=0.5)


mask_shape = gv.Shape.from_shapefile(shapefile)[0]
vertex_stream = PolyEdit(source=mask_shape)
tiles * mask_shape

### For more information:

[http://earthsim.pyviz.org/index.html](http://earthsim.pyviz.org/index.html)